In [1]:
raw_ncar_folder_path = '/home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt/'
save_folder_path = '/home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_reading/visualizations/'

var = 'p'

# timestep_range = range(5) # This will definitely crash the Kernel
timestep_nr = 2

In [2]:
# !pip install "xarray[complete]"

In [3]:
# needed for sciserver jobs
%cd /home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_reading

/home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_reading


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [5]:
# for timestep_nr in timestep_range:
#     data = xr.open_dataset(raw_ncar_folder_path + "/jhd.00" + str(timestep_nr) + ".nc")


data_xr = xr.open_dataset(raw_ncar_folder_path + "/jhd.00" + str(timestep_nr) + ".nc")
# data = data['e']

In [6]:
steps = range(0, data_xr['e'].shape[2], 10)

# for i, z in enumerate(z_steps):
#     print(i,z)

In [ ]:
data = np.array(data_xr[var])

### Runtimes

3min 45s +- 4min(!) to load Array `np.array(data[var])`

33.3s to plot using in-memory numpy array (from above)

14min 17s +- 6min for loading data using `data_xr[var][slice]` every time

Skipping the np.array loading takes wayyy too long (>1h for 7 runs atm)

In [ ]:
for y in steps:#[0]:
    slice_data = data[:, y, :] # Change this to slice along a different axis
    plt.figure(figsize=(20,20))
    plt.imshow(slice_data, cmap='gist_gray')#, vmin=-.5, vmax=.6)

#     name = "[z,y,x=" + str(x) + "]" # x
    name = "[z,y=" +str(y) + ",x]" # y
    
    plt.title(name, fontsize=20)

    plt.savefig(f'visualizations/timestep_' + str(timestep_nr) + '/' + var + "/" + name + '.png')
    plt.close()

del(data)

## Create movie

In [9]:
%cd visualizations

/home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_reading/visualizations


In [11]:
vars = list(data_xr.variables)
vars

['u', 'v', 'w', 't', 'p', 'e']

In [13]:
# Create GIF from PNG files

from PIL import Image
import os

for var in vars:
    png_directory = 'timestep_1/' + var
    png_files = [file for file in os.listdir(png_directory) if file.endswith('.png')]
    png_files.sort()

    frames = []

    for png_file in png_files:
        png_path = os.path.join(png_directory, png_file)
        image = Image.open(png_path)
        frames.append(image)

    output_gif_path = png_directory + '/movie.gif'
    frames[0].save(output_gif_path, save_all=True, append_images=frames[1:], duration=200, loop=0)

In [ ]:
from PIL import Image
import os

# Get a list of folders named timestep_0 to timestep_4
timestep_folders = [folder for folder in os.listdir('.') if folder.startswith('timestep_') and os.path.isdir(folder)]

# Get a list of subfolders inside the first timestep folder (assuming all timestep folders have the same subfolders)
first_timestep_folder = os.path.join(timestep_folders[0])
subfolders = [subfolder for subfolder in os.listdir(first_timestep_folder) if os.path.isdir(os.path.join(first_timestep_folder, subfolder))]

for timestep_folder in timestep_folders:
    for subfolder in subfolders:
        png_directory = os.path.join(timestep_folder, subfolder)

        png_files = [file for file in os.listdir(png_directory) if file.endswith('.png')]
        png_files.sort()

        frames = []

        for png_file in png_files:
            png_path = os.path.join(png_directory, png_file)
            image = Image.open(png_path)
            frames.append(image)

        output_gif_path = os.path.join(png_directory, 'movie.gif')
        frames[0].save(output_gif_path, save_all=True, append_images=frames[1:], duration=200, loop=0)

print("GIFs created successfully!")
